In [12]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline ,make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier


In [2]:
path = 'C:/Users/himan/OneDrive/Desktop/Titanic-Dataset.csv'

df = pd.read_csv(path)

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
#Droping columns which are of no use.
df.drop(columns=['Name','PassengerId','Ticket','Cabin'],inplace=True)

In [5]:
#step-1 ------> train test split
# spliting depandent and indepandent varibles. 
x = df.iloc[:,1:]
y = df.iloc[:,0]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)



In [6]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [7]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [8]:
#step  first  imputation transformer

tran1 = ColumnTransformer([
        ('impute_age',SimpleImputer(),[2]), # felling mean at index 2 ['Sex']
        ('impute_emb',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')



In [9]:
# step2
#OneHotEncoding
tran2 = ColumnTransformer([
        ('ohe_sex_emb',OneHotEncoder(sparse = False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [10]:
## scaling

tran3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [13]:
#feature Scaling

tran4 = SelectKBest(score_func=chi2,k=8)

In [14]:
#train MOdel

tran5 =  DecisionTreeClassifier()

## Creating Pipelines

In [15]:
Pipe = Pipeline([
    ('tran1',tran1),
    ('tran2',tran2),
    ('tran3',tran3),
    ('tran4',tran4),
    ('tran5',tran5)
])

In [17]:
# train
Pipe.fit(x_train,y_train)


Pipeline(steps=[('tran1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_emb',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tran2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_emb',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('tran3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
       

In [18]:
from sklearn import set_config
set_config(display='diagram')

In [19]:
Pipe.fit(x_train,y_train)


Pipeline(steps=[('tran1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_emb',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tran2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_emb',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('tran3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tran4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001E61373A820>)),
                ('tran5', DecisionTreeClassifier())])

In [20]:
Pipe.named_steps

{'tran1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_emb',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tran2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_emb',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'tran3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'tran4': SelectKBest(k=8, score_func=<function chi2 at 0x000001E61373A820>),
 'tran5': DecisionTreeClassifier()}

In [26]:
Pipe.named_steps['tran1'].transformers_[0][1].statistics_ # these are mean values while imputing

array([29.49884615])

In [27]:
Pipe.named_steps['tran1'].transformers_[1][1].statistics_ # most frequent 

array(['S'], dtype=object)

In [28]:
#predict

y_pred = Pipe.predict(x_test)

In [30]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred) ## low because of feature scaling

0.6256983240223464

In [32]:
import pickle
pickle.dump(Pipe,open('pipe.pk1','wb'))